# Libraries

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image
from tqdm.auto import tqdm
from shutil import copyfile
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
! pip install plot_keras_history silence_tensorflow
from plot_keras_history import plot_history
#import silence_tensorflow.auto

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ( 
    EarlyStopping, 
    ReduceLROnPlateau, 
    ModelCheckpoint, 
    BaseLogger, 
    TerminateOnNaN,
)
from tensorflow.keras.metrics import AUC

cudnn fails to initialize if we don't include the following cell `¯\_(ツ)_/¯`

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.compat.v1.Session(config=config)

# Download and unzip the data

In [1]:
#! curl -L -o "archive.zip" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/23921/1664239/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1605518469&Signature=WdCw3%2FNSWvK2uzLV7mPLRwsz3JM5cnC3kAOc3%2BDqILc3sdWW%2BqmFcwuVSBHA3wACn4%2Fxa5LaheDYQRQHg58T2YjPw7IbtiuUUY2RUKYRB1xsY7VUzP2LocsA%2F63QRbYhwGFTzGm9ExyZA8axnUHVzbs9TZ5sJXXQnj0u4cbBEPt%2FFSbnJ6C971LlJhXRk%2F5124kMjtyH1ps4o%2BLmmcmYZ7E838Fb7yP9WL3whXIKpT9pSrR2Sgk3%2FGrj727wjwswK75mNfJv9fZWfMA1dgJiogmmW1ijLyGZ1woGCG49k5npf%2FtHdtcf40BKHX1KiDOQko4HTevbnseSquQ%2FOqpvqA%3D%3D&response-content-disposition=attachment%3B+filename%3Dartificial-neural-networks-and-deep-learning-2020.zip" 2>1 > /dev/null

In [2]:
#! unzip archive.zip

In [ ]:
#! ls

# Hyper-parameters and settings

In [ ]:
SEED = 0xc0febabe
BATCHSIZE = 16
IMAGE_SHAPE = (256, 256, 3)

In [ ]:
CHECKPOINTS_FOLDER = "./weights/checkpoints_{}/".format(SEED)
PROCESSED_IMAGES_FOLDER = "./SPLIT_{}".format(SEED)

In [ ]:
# Create the folders
_ = list(map(
    lambda folder: os.makedirs(folder, exist_ok=True), 
    [CHECKPOINTS_FOLDER, PROCESSED_IMAGES_FOLDER]
))

In [ ]:
# set the seeds for reproducibility
tf.random.set_seed(SEED) 
np.random.seed(SEED)

# Setup the validation by splitting the data in sub-folders

Label -> folder dictionary

In [ ]:
labels_dir = {
    0:"0_NO_PERSON",
    1:"1_ALL_THE_PEOPLE",
    2:"2_SOMEONE"
}

load the labels

In [ ]:
with open("./MaskDataset/train_gt.json") as f:
    labels = json.load(f)

use a stratified shuffle split to have a 90-10 split which matains the balancing of the data

In [ ]:
labels_list = np.array(list(labels.items()))
sss = StratifiedShuffleSplit(test_size=0.1, random_state=SEED)
train_indices, val_indices = next(sss.split(labels_list[:, 0], labels_list[:, 1]))
train_files, train_labels = labels_list[train_indices][:, 0], labels_list[train_indices][:, 1]
val_files, val_labels = labels_list[val_indices][:, 0], labels_list[val_indices][:, 1]

Copy the files in folders based on their label

In [ ]:
def process(files, labels, dst_folder):
    for file_name, label in tqdm(zip(files, labels), leave=False):
        file = os.path.join("./MaskDataset/training/", file_name)
        label = labels_dir[int(label)]
        dst_file = os.path.join(
            dst_folder,
            label,
            file_name
        )
        os.makedirs(os.path.dirname(dst_file), exist_ok=True)
        copyfile(file, dst_file)

In [ ]:
train_folder = os.path.join(PROCESSED_IMAGES_FOLDER, "train")
val_folder = os.path.join(PROCESSED_IMAGES_FOLDER, "val")

In [ ]:
train_folder, val_folder

In [ ]:
process(
    train_files,
    train_labels,
    train_folder
)
process(
    val_files,
    val_labels,
    val_folder
)

Create a dataset with the images and augment them

In [ ]:
def preprocess_func(img):
    img = tf.image.random_saturation(img, 0.8, 1.2)
    img = tf.image.random_hue(img, 0.1)
    img = tf.image.random_contrast(img, 0.8, 1.2)
    img = tf.image.random_brightness(img, 0.2)
    return img

train_data_gen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.25,
        height_shift_range=0.25,
        zoom_range=0.1, 
        shear_range=0.2,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='constant',
        rescale=1./255,
        preprocessing_function=preprocess_func
    )
    
train_gen = train_data_gen.flow_from_directory(
    train_folder,
    batch_size=BATCHSIZE,
    target_size=IMAGE_SHAPE[:-1],
    class_mode='categorical',
    shuffle=True,
    seed=SEED,
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, *IMAGE_SHAPE], [None, len(labels_dir)])
).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
val_data_gen = ImageDataGenerator(rescale=1./255)

val_gen = val_data_gen.flow_from_directory(
    val_folder,
    batch_size=BATCHSIZE,
    target_size=IMAGE_SHAPE[:-1],
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)
    
val_dataset = tf.data.Dataset.from_generator(
    lambda: val_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, *IMAGE_SHAPE], [None, 3])
).prefetch(tf.data.experimental.AUTOTUNE)

# Create the model

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2

truncated = InceptionResNetV2(
    input_shape=IMAGE_SHAPE, 
    include_top=False,
    weights="imagenet"
)

In [ ]:
i = truncated.input
h = truncated.output

h = Flatten()(h)
h = Dense(100, activation="linear", 
    kernel_initializer=tf.keras.initializers.GlorotNormal()
)(h)
h = Dropout(0.5)(h)
h = BatchNormalization()(h)
h = Activation("relu")(h)
h = Dense(10, activation="linear", 
    kernel_initializer=tf.keras.initializers.GlorotNormal()
)(h)
h = BatchNormalization()(h)
h = Activation("relu")(h)
h = Dropout(0.2)(h)
output = Dense(3, activation="softmax", 
    kernel_initializer=tf.keras.initializers.GlorotNormal()
)(h)

model = Model(i, output)

In [ ]:
model.summary()
#plot_model(model)

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="nadam",
    metrics=[
        "accuracy",
        AUC(curve="PR", name="AUPRC", multi_label=True),
        AUC(curve="ROC", name="AUROC", multi_label=True),
    ],
)

train the model

In [ ]:
checkpoints_folder = os.path.join(
    CHECKPOINTS_FOLDER,
    datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
)
os.makedirs(checkpoints_folder, exist_ok=True)
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1,
    steps_per_epoch=512,
    validation_steps=16,
    callbacks=[
        EarlyStopping(
            monitor="val_loss",
            min_delta=0.001,
            patience=20,
            restore_best_weights=True
        ),
        ReduceLROnPlateau(
            monitor="val_loss",
            min_delta=0.001,
            patience=5,
            factor=0.1,
        ),
        ModelCheckpoint(
            checkpoints_folder,
            monitor="val_loss",
            mode="max",
            save_weights_only=True
        ),
        TerminateOnNaN(),
    ]
    
).history

visualize the model training

In [ ]:
plot_history(history)

we save the weighs

In [ ]:
model.save_weights("validation_final.h5")

and evaluate the final performance

In [ ]:
model.evaluate(val_dataset, steps=10)

In [ ]:
model.evaluate(train_dataset, steps=100)

# Compute the predictions

In [ ]:
result = {}
keys = []
imgs = []

for file in tqdm(glob("./MaskDataset/test/*"), leave=False):
    key = os.path.basename(file)
    keys.append(key)
    img = Image.open(file).convert('RGB').resize(IMAGE_SHAPE[:-1])
    img = np.array(img).reshape(1, *IMAGE_SHAPE)/255.0
    imgs.append(img)

In [ ]:
prediction = model.predict(np.vstack(imgs), batch_size=16)

In [ ]:
label = np.argmax(prediction, axis=1)
result = list(zip(keys, label))    

df = pd.DataFrame(result, columns=["Id", "Category"])
df = df.set_index("Id")
df.to_csv("predictions_NUOVE.csv")
df